
<h1 id="Tutorial-3---Hairpin-transitions">Tutorial 3 - Hairpin transitions<a class="anchor-link" href="#Tutorial-3---Hairpin-transitions">¶</a></h1>



<p>Having seen how the simulation generates trajectories in Tutorial 2 -  Hairpin Trajectories, here we extract some information from the trajectories.  Using the example of Chapter 7.3 of Schaeffer's PhD thesis, this script shows how to use macrostate definitions and Transition Mode to extract a coarse-grained description of the system's motion through state space.</p>
<p>This mode requires a little more post-processing of the Multistrand simulations... but don't be put off.  It's easy.</p>
<p>Note that the transition times listed here are briefer than those that appear in the tables of Chapter 7.3 of Schaeffer's thesis.  That is because we are using different unimolecular and bimolecular rate constant values.</p>


In [1]:
from multistrand.objects import *
from multistrand.options import Options, Literals
from multistrand.system import SimSystem
import numpy as np


<p>For <code>StopCondition</code> and <code>Macrostate</code> definitions:</p>



<h4 id="Setup-options-for-simulation">Setup options for simulation<a class="anchor-link" href="#Setup-options-for-simulation">¶</a></h4>


In [2]:
def setup_options_hairpin():
    """
    setup_options_hairpin( )

    Returns the options object for simple hairpin example of
    transition mode in Chapter 7.3 of Schaeffer's PhD thesis.
    """
    # Once domains are defined, strands can be built from them using "+".
    stem = Domain(name="stem",sequence="GCATGC",length=6)
    hp = Domain(name="hairpin", sequence="AAAA",length=4)
    s = stem + hp + stem.C
    
    # Note that because we defined domains, we can either represent secondary structures at the domain level or at the sequence level.
    start_complex = Complex(strands=[s], structure="...")
    pathway_endside_exact_complex = Complex(strands=[s], structure="(((..........)))")
    pathway_endside_loose_complex = Complex(strands=[s], structure="(((**********)))")
    pathway_hpside_exact_complex = Complex(strands=[s],  structure="...(((....)))...")
    pathway_hpside_loose_complex = Complex(strands=[s],  structure="***(((****)))***")
    full_complex = Complex( strands=[s], structure="(.)")

    # Define macrostates to be tracked, i.e. we look for transitions between them.
    initial_sc           = Macrostate("INITIAL", [(start_complex,Literals.exact_macrostate,0)])
    pathway_hp_exact_sc  = Macrostate("HPSIDE_EXACT", [(pathway_hpside_exact_complex,Literals.exact_macrostate,0)])
    pathway_hp_loose_sc  = Macrostate("HPSIDE_LOOSE", [(pathway_hpside_loose_complex,Literals.loose_macrostate,2)])   # within distance 2 
    pathway_end_exact_sc = Macrostate("ENDSIDE_EXACT", [(pathway_endside_exact_complex,Literals.exact_macrostate,0)])
    pathway_end_loose_sc = Macrostate("ENDSIDE_LOOSE", [(pathway_endside_loose_complex,Literals.loose_macrostate,2)]) # within distance 2 
    full_sc              = StopCondition("stop:FULL", [(full_complex,Literals.exact_macrostate,0)])
    # Multistrand treats Macrostates and StopConditions interchangeably; here we choose the name as a mnemonic for how it will be used.
    # The simulation will stop the first time that 'full_sc' is reached, so we call it a StopCondition.
    # The simulation keeps track of where it is, but keeps going, when it visits the others -- so they are called Macrostates.
    # But the simulation would proceed identically if we randomly called some Macrostates and others StopConditions.  
    # What determines Multistrand's behavior is that the name of 'full_sc' begins with "stop:" --
    # this is what causes the simulation to stop when 'full_sc' is reached.

    # We will set up two Transition Mode simulations, one looking at transitions between exact states...
    o_exact = Options(simulation_mode="Transition", substrate_type="DNA", temperature=310.15,
                      num_simulations=1000, simulation_time=.01, start_state=[start_complex], verbosity=0)
    o_exact.stop_conditions = [initial_sc, pathway_end_exact_sc, pathway_hp_exact_sc, full_sc]
    o_exact.JSMetropolis37()

    # ... and one looking at transitions between loosely defined macrostates
    o_loose = Options(simulation_mode="Transition",parameter_type="Nupack",substrate_type="DNA", temperature=310.15,
                      num_simulations=1000, simulation_time=.01, start_state=[start_complex], verbosity=0)
    o_loose.stop_conditions = [initial_sc, pathway_end_loose_sc, pathway_hp_loose_sc, full_sc]
    o_loose.JSMetropolis37()
    
    # change verbosity to 1, above, and you'll see a print-out during the simulation runs, every time a stop state is reached.
    return o_exact, o_loose


<h4 id="Some-helper-code-follows.">Some helper code follows.<a class="anchor-link" href="#Some-helper-code-follows.">¶</a></h4><p>The following naming routines are motivated by the observation that when "loose" macrostates are used, it is possible that a given system state may belong to more than one macrostate.  Thus, transitions are not between macrostates per se, but are between macrostate membership vectors.  I.e. if the macrostates are A, B, C, and D, then a single simulation step might take the system state from being in both A and C, to being in B, C, and D.  And perhaps thence to being in no macrostates.</p>


In [3]:
# mol will be a list of True/False for which transition macrostates the system has entered,
# so in_state(mol) returns True if the system is in at least one of the listed macrostates.
def in_state(mol): return sum(mol) > 0

# mol is a Boolean descriptor of macrostate occupancy, like mol above.
# a short-hand name for this macrostate (based on the order given in stop_conditions) is provided.
def mol_name(mol):
    charindex = 'ABCDEFGHIJKLMNOPQRSTUVWXYZ0'
    names = [charindex[j] for j,i in enumerate(mol) if i]
    if names == []:
        names = charindex[26]
    else:
        names = ",".join(names)
    return names

# t0 and t1 are Boolean descriptors of macrostate occupancy, like mol above.
# here, we provide a printable name for the transition between two macrostate occupancy lists.
def trans_name(t0,t1):
    return mol_name(t0) + ' -> ' + mol_name(t1)

def print_transitions(transition_traj):
    for t in transition_traj:
        print(f"{t[0]:12g} : {mol_name(t[1])}")
          
# for each simulation, the transition trajectory reports the tuple (time_entered, which_macrostates_the_system_is_now_in)
def parse_transition_lists(transition_traj_list):
    transition_dict = {}

    # the mol1 --> mol2 transition times represent (time of entry into mol1) to (time of entry into mol2)
    for transition_traj in transition_traj_list:
        truncated = [i for i in transition_traj if in_state(i[1])]
        tt = truncated # only keep the entry times and mol states for non-trivial mols

        for i in range(len(tt)-1):
            nm = trans_name(tt[i][1],tt[i+1][1])
            if nm in transition_dict:
                transition_dict[nm].append(tt[i+1][0] - tt[i][0])
            else:
                transition_dict[nm] = [tt[i+1][0] - tt[i][0]]

    return transition_dict

def parse_transition_list(transition_traj_list):
    return parse_transition_lists([transition_traj_list])

    
def print_transition_dict(transition_dict, options=None):
    k = list(transition_dict.keys())
    k.sort()

    for i in k:
        transition_times = np.array(transition_dict[i])
        print(f"{i}: {np.mean(transition_times):.2e} ({len(transition_dict[i])})")

    # also print the true names of the macrostates, if an Options object is provided
    charindex = 'ABCDEFGHIJKLMNOPQRSTUVWXYZ0'
    if options:
        for idx, i in enumerate(options.stop_conditions):
            print(f"{i.tag}: {charindex[idx]}")


<h4 id="Back-to-stuff-to-try...-automatically-or-by-hand,-line-by-line">Back to stuff to try... automatically or by hand, line-by-line<a class="anchor-link" href="#Back-to-stuff-to-try...-automatically-or-by-hand,-line-by-line">¶</a></h4>


In [4]:
print("--- Running Simulations ---")
o_exact, o_loose = setup_options_hairpin()
s = SimSystem(o_exact)
s.start()
s = SimSystem(o_loose)  
s.start()
print("--- Finished simulations ---")

--- Running Simulations ---
--- Finished simulations ---


In [5]:
def print_results(o, label: str):
    print()
    print(f"--- Analysis of simulations with {label} transition states ---")
    print("  Coarse-grained trajectory of simulation #1:")
    print_transitions(o.interface.transition_lists[0])
    print("  Transitions from simulation #1:")
    parsedlist = parse_transition_list(o.interface.transition_lists[0])
    print_transition_dict(parsedlist)
    print("  Transitions averaged over all %d simulations:" % o.num_simulations)
    parsedlist = parse_transition_lists(o.interface.transition_lists)
    print_transition_dict(parsedlist, o) # adds names for macrostates

In [6]:
print_results(o_exact, "exact")
print_results(o_loose, "loose")


--- Analysis of simulations with exact transition states ---
  Coarse-grained trajectory of simulation #1:
           0 : A
 8.78574e-10 : 0
 1.40498e-09 : A
 4.49546e-09 : 0
 6.84786e-09 : A
 8.85494e-09 : 0
 9.31992e-09 : A
 9.33026e-09 : 0
  1.0786e-08 : A
 1.37033e-08 : 0
 1.38448e-08 : A
 1.52093e-08 : 0
 1.61841e-08 : A
 1.74344e-08 : 0
 9.31578e-07 : A
 9.33543e-07 : 0
 9.34899e-07 : A
 9.37078e-07 : 0
 9.37275e-07 : A
 9.37333e-07 : 0
 9.44343e-07 : A
 9.46842e-07 : 0
 9.49417e-07 : A
 9.50396e-07 : 0
 9.52385e-07 : A
 9.54505e-07 : 0
  9.5474e-07 : A
 9.55257e-07 : 0
 9.57204e-07 : A
 9.57512e-07 : 0
  9.5968e-07 : A
 9.59684e-07 : 0
 9.59821e-07 : A
 9.64028e-07 : 0
 9.64095e-07 : A
 9.64473e-07 : 0
 9.66533e-07 : C
 9.67112e-07 : 0
 9.68867e-07 : D
  Transitions from simulation #1:
A -> A: 5.67e-08 (17)
A -> C: 2.44e-09 (1)
C -> D: 2.33e-09 (1)
  Transitions averaged over all 1000 simulations:
A -> A: 1.84e-08 (9482)
A -> B: 5.43e-09 (140)
A -> C: 3.16e-09 (511)
A -> D: 5.7


<h4 id="A-final-thought">A final thought<a class="anchor-link" href="#A-final-thought">¶</a></h4><p>Ponder why, for exact macrostates, the A -&gt; D transition can occur and is not even rare, while with loose macrostates, A transitions exclusively either to B or to C.</p>
